In [1]:
import numpy_financial as npf

In [2]:
def monthly_PI(interest, years, price, yearly_months=12):
    monthly_PI = -1 * npf.pmt(interest/yearly_months, years*yearly_months, price)
    return monthly_PI

In [21]:
# purchase
purchase_price = 600e3
downpayment = 21e3
rehab_cost = 100e3
after_repair_value = 800e3

# acquisition
yearly_months = 12
acq_yearly_interest = 0 # 0.065
mortgage_years = 30
rehab_months = 6
monthly_taxes = 555 # from zillow

# # initial rental period
monthly_rent = 4e3
vacancy_frac, repairs_frac, capex_frac = 0.05, 0.05, 0.05

# refinanced rental period 
refinance_months = 9
pre_refi_duration = refinance_months-rehab_months

In [4]:
class Mortgage():
    def __init__(rate, years, price):
        self.org_monthly_PI = self.monthly_PI(rate, years, price)
    def original_contract(self, rate, years, price, yearly_months=yearly_months):
        monthly_PI = -1 * npf.pmt(rate/yearly_months, years*yearly_months, price)
        return monthly_PI

In [72]:
class Acquisition():
    """ Could inherent a Mortgage object if it could contain several methods """
    def __init__(self, purchase_price, downpayment, yearly_interest):
        self.time = {}
        self.price = {
            'home_value': purchase_price,
            'downpayment': downpayment,
            'mortgage': np.nan,
            'monthly_PI': np.nan,
            'monthly_taxes': monthly_taxes,
            
        } 
        self.exponent ={
            'yearly_interest': yearly_interest,
        }
        
        self.derive_properties()
        
    def derive_properties(self):
#         super().__init__(self.price['yearly_interest'], 30, self.price['price'])
        self.price['loan_fees'] = 0.01 * self.price['home_value']
        self.price['mortgage'] = self.price['home_value'] - self.price['downpayment'] + self.price['loan_fees']
        self.price['monthly_PI'] = monthly_PI(self.exponent['yearly_interest'], 30, self.price['mortgage'])
        self.price['closing'] = self.price['home_value'] * 0.01 
        
    def __str__(self):
        return (
            '\nAcquisition:\n'
            f"Down Payment: ${acq.price['downpayment']}\n"
            f"Loan Amount: ${acq.price['mortgage']}\n"
            f"Loan Points/Fees: ${acq.price['loan_fees']}\n"
            f"Loan Interest Rate: {acq.exponent['yearly_interest']*100}%\n"
            f"Monthly P&I: ${acq.price['monthly_PI']:.2f}"
         )
        
class Rehab():
    def __init__(self, rehab_months, total_cost, monthly_PI, monthly_insurance=0, other_costs = 0):
        self.time = {
            'total_months' : rehab_months
        }
        self.price= {
            'total_cost': total_cost,
            'monthly_insurance': monthly_insurance,
            'other': other_costs,
            'holding_cost': np.nan,
            'monthly_PI': monthly_PI,
            'monthly_taxes': monthly_taxes
        } 
        self.exponent = {
        }

        self.derive_properties()
        
    def derive_properties(self):
        self.price['monthly_rehab'] = self.price['total_cost']*self.time['total_months']/yearly_months
        self.price['holding_cost'] = (self.price['monthly_PI'] + self.price['monthly_taxes']) * self.time['total_months']
        self.price['monthly_total'] = self.price['monthly_rehab'] + self.price['monthly_taxes'] + self.price['monthly_PI']
        
    def __str__(self):
        return (
            '\nRehab:\n'
            f"Holding costs: ${self.price['holding_cost']:.2f}\n"
            f"Monthly P&I: ${self.price['monthly_PI']:.2f}\n"
            f"Rehab time: {self.time['total_months']} months\n"
            f"Monthly taxes: ${self.price['monthly_taxes']}"
        )
    
class PreReFi_Rent():
    """ Could inherent a Mortgage object if """
    def __init__(self, monthly_rent, vacancy_frac, repairs_frac, capex_frac, total_time, 
                monthly_PI):
        self.time = {
            'total': total_time
        }
        self.price =  {
            'monthly_rent': monthly_rent,
            'vacancy_frac': vacancy_frac,
            'repairs_frac': repairs_frac,
            'capex_frac': capex_frac,
            'monthly_taxes': monthly_taxes,
            'monthly_PI': monthly_PI
        } 
        self.exponent = {}
        self.derive_properties()
        
    def derive_properties(self):
        self.price['monthly_vacancy'] = self.price['monthly_rent']*self.price['vacancy_frac']
        self.price['monthly_repairs'] = self.price['monthly_rent']*self.price['repairs_frac']
        self.price['monthly_capex'] = self.price['monthly_rent']*self.price['capex_frac']
        self.price['monthly_expenses'] = self.sum_expenses()
        self.price['monthly_cashflow'] = self.price['monthly_rent'] - self.price['monthly_expenses']
        self.price['NOI'] = (self.price['monthly_cashflow'] + self.price['monthly_PI']) * yearly_months  # googled NOI vs cash flow

    def sum_expenses(self):
        exp_sum = self.price['monthly_vacancy'] + self.price['monthly_capex'] \
        + self.price['monthly_taxes'] + self.price['monthly_repairs'] + self.price['monthly_PI']
        return exp_sum
    
    def __str__(self):
        return (
            '\nInitial Rental Period:\n'
            f"Monthly Income: ${self.price['monthly_rent']}\n"
            f"Monthly Expenses: ${self.price['monthly_expenses']:.2f}\n"
            f"Monthly Cashflow: ${self.price['monthly_cashflow']:.2f}\n"
            f"NOI: ${self.price['NOI']}"
        )

class Refinance():
    """ Could inherent a Mortgage object if """
    def __init__(self, monthly_rent, home_value, vacancy_frac, repairs_frac, capex_frac):
        self.time =  {}
        self.price =  {
            'monthly_rent': monthly_rent,
            'home_value': home_value,
            'vacancy_frac': vacancy_frac,
            'repairs_frac': repairs_frac,
            'capex_frac': capex_frac,
            'monthly_PI': np.nan,
            'monthly_taxes': monthly_taxes
        } 
        self.exponent = {
            'yearly_interest': 0#0.065
        }
        self.derive_properties()

    def derive_properties(self):
        self.price['monthly_vacancy'] = self.price['monthly_rent']*self.price['vacancy_frac']
        self.price['monthly_repairs'] = self.price['monthly_rent']*self.price['repairs_frac']
        self.price['monthly_capex'] = self.price['monthly_rent']*self.price['capex_frac']
        self.price['loan_fees'] = 0.01 * self.price['home_value']
        self.price['mortgage'] = 0.8 * self.price['home_value'] + self.price['loan_fees']
        self.price['monthly_PI'] = monthly_PI(self.exponent['yearly_interest'], 30, self.price['mortgage'])
        self.price['monthly_expenses'] = self.sum_expenses()
        self.price['monthly_cashflow'] = self.price['monthly_rent'] - self.price['monthly_expenses']
        self.price['NOI'] = (self.price['monthly_cashflow'] + self.price['monthly_PI']) * yearly_months  # googled NOI vs cash flow

    def sum_expenses(self):
        exp_sum = self.price['monthly_vacancy'] + self.price['monthly_capex'] \
        + self.price['monthly_taxes'] + self.price['monthly_repairs'] + self.price['monthly_PI']
        return exp_sum
    
    def __str__(self):
        return (
            '\nRefinance:\n'
            f"Loan Amount: ${refi.price['mortgage']}\n"
            f"Loan Points/Fees: ${refi.price['loan_fees']}\n"
            f"Loan Interest Rate: {refi.exponent['yearly_interest']*100}%\n"
            f"Monthly P&I: ${refi.price['monthly_PI']:.2f}"
        )

In [73]:
acq = Acquisition(purchase_price, downpayment, acq_yearly_interest)
rehab = Rehab(rehab_months, rehab_cost, acq.price['monthly_PI'])
pre_refi = PreReFi_Rent(monthly_rent, vacancy_frac, repairs_frac, capex_frac, 
                        pre_refi_duration, acq.price['monthly_PI'])
refi = Refinance(monthly_rent, after_repair_value, vacancy_frac, repairs_frac, capex_frac)

In [74]:
print(str(acq))
print(str(rehab))
print(str(pre_refi))
print(str(refi))


Acquisition:
Down Payment: $21000.0
Loan Amount: $585000.0
Loan Points/Fees: $6000.0
Loan Interest Rate: 0%
Monthly P&I: $1625.00

Rehab:
Holding costs: $13080.00
Monthly P&I: $1625.00
Rehab time: 6 months
Monthly taxes: $555

Initial Rental Period:
Monthly Income: $4000.0
Monthly Expenses: $2780.00
Monthly Cashflow: $1220.00
NOI: $34140.0

Refinance:
Loan Amount: $648000.0
Loan Points/Fees: $8000.0
Loan Interest Rate: 0%
Monthly P&I: $1800.00


In [27]:
cash_required = acq.price['downpayment'] + rehab.price['total_cost'] + acq.price['closing']
cash_outlay = cash_required + rehab.price['holding_cost']
acq_cash_on_cash_roi = pre_refi.price['monthly_cashflow']*yearly_months/cash_required
ref_cash_on_cash_roi = refi.price['monthly_cashflow']*yearly_months/cash_required

pro_forma_cap_rate = pre_refi.price['NOI']/refi.price['mortgage']
purchase_cap_rate = pre_refi.price['NOI']/acq.price['mortgage']